In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# import files
ad_feature = '/content/drive/MyDrive/datasets_Aug12/ad_feature.csv'
behavior_log = '/content/drive/MyDrive/datasets_Aug12/behavior_log.csv'
raw_sample = '/content/drive/MyDrive/datasets_Aug12/raw_sample.csv'
user_profile = '/content/drive/MyDrive/datasets_Aug12/user_profile.csv'

In [ ]:
user_data = pd.read_csv(user_profile)
user_data.head()
print(user_data.shape)
user_data.head()

ads = pd.read_csv(ad_feature)
ads.head()
print(ads.shape)
ads.head()

raw_sample = pd.read_csv(raw_sample, nrows=2000000)
raw_sample.to_csv('/content/drive/MyDrive/datasets_Aug12/sample_data.csv', index=False, sep=',')
dataset = pd.read_csv('/content/drive/MyDrive/datasets_Aug12/sample_data.csv')
print(dataset.shape)
dataset.head()

(1061768, 9)
(846811, 6)
(2000000, 6)


,user,time_stamp,adgroup_id,pid,nonclk,clk
0,581738,1494137644,1,430548_1007,1,0
1,449818,1494638778,3,430548_1007,1,0
2,914836,1494650879,4,430548_1007,1,0
3,914836,1494651029,5,430548_1007,1,0
4,399907,1494302958,8,430548_1007,1,0


In [ ]:
user_data_null = user_data.isnull().sum()/len(user_data)*100
user_data_null = user_data_null.drop(user_data_null[user_data_null==0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio(%)':user_data_null})
print(f'user_data含有缺失值的属性个数：{len(user_data_null)}')
print(missing_data)
print('-' * 60)

ads_null = ads.isnull().sum()/len(ads)*100
ads_null = ads_null.drop(ads_null[ads_null==0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio(%)':ads_null})
print(f'ads含有缺失值的属性个数：{len(ads_null)}')
print(missing_data)
print('-' * 60)

dataset_null = dataset.isnull().sum()/len(dataset)*100
dataset_null = dataset_null.drop(dataset_null[dataset_null==0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio(%)':dataset_null})
print(f'dataset含有缺失值的属性个数：{len(dataset_null)}')
print(missing_data)


user_data含有缺失值的属性个数：2
                       Missing Ratio(%)
pvalue_level                  54.241322
new_user_class_level          32.485439
------------------------------------------------------------
ads含有缺失值的属性个数：1
       Missing Ratio(%)
brand         29.089136
------------------------------------------------------------
dataset含有缺失值的属性个数：0
Empty DataFrame
Columns: [Missing Ratio(%)]
Index: []


In [ ]:
# 对 'new_user_class_level '进行众数填充
from sklearn.impute import SimpleImputer   # 导入缺失值处理所需的库impute.SimpleImputer
# 使用reshape(-1,1)对数据升维，原本是一维，但在sklearn当中的特征矩阵必须是二维的
city_level = user_data.loc[:,'new_user_class_level '].values.reshape(-1,1)  # loc：对索引名进行切片
si = SimpleImputer(strategy = 'most_frequent')  # 实例化，使用众数填补
user_data.loc[:,'new_user_class_level '] = si.fit_transform(city_level) # fit_transform一步训练导出结果
user_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061768 entries, 0 to 1061767
Data columns (total 9 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   userid                 1061768 non-null  int64  
 1   cms_segid              1061768 non-null  int64  
 2   cms_group_id           1061768 non-null  int64  
 3   final_gender_code      1061768 non-null  int64  
 4   age_level              1061768 non-null  int64  
 5   pvalue_level           485851 non-null   float64
 6   shopping_level         1061768 non-null  int64  
 7   occupation             1061768 non-null  int64  
 8   new_user_class_level   1061768 non-null  float64
dtypes: float64(2), int64(7)
memory usage: 72.9 MB


In [ ]:
# 调整数据集user_data的列顺序：将'pvalue_level'列调至最后一列
columns = ['userid', 'cms_segid', 'cms_group_id', 'final_gender_code', 'age_level',
       'shopping_level', 'occupation', 'new_user_class_level ','pvalue_level']
user_data = user_data[columns]
"""
按'pvalue_level'列值是否为空对数据集user_data进行分离
"""
# 'pvalue_level'列值为空
pvalue_null = user_data.loc[user_data['pvalue_level'].isnull().values == True]
# 'pvalue_level'列值为空
pvalue_nonull = user_data.loc[user_data['pvalue_level'].isnull().values == False]
"""
对数据集pvalue_null和pvalue_nonull
分离训练集和测试集
X_train_user：'pvalue_level'列以外的，且'pvalue_level'列值不为0的部分
y_train_user：'pvalue_level'列中的，且值不为0
X_test_user：'pvalue_level'列以外的，且值为0
y_test_user：'pvalue_level'列中的，且值为0
"""
X_train_user,y_train_user = pvalue_nonull.iloc[:,:-1],pvalue_nonull.iloc[:,-1]
X_test_user,y_test_user = pvalue_null.iloc[:,:-1],pvalue_null.iloc[:,-1]
# 运用K最近邻（k-Nearest Neighbor，KNN）分类算法对'pvalue_level'列预测
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3,weights='distance')  # 根据消费档次1浅2中3深分为3类，weights='distance'表示用欧氏距离进行相似度衡量
knn.fit(X_train_user,y_train_user)   # 训练数据集
y_test_user = knn.predict(X_test_user)   # 导出结果
y_test_user   # 输出结果为：array([2., 2., 2., ..., 1., 2., 2.])

"""
对KNN算法预测的数据进行整理合并填充，将整理后的数据写入新表user
"""
y_test_user = pd.DataFrame(y_test_user)   # 将y_test_user由array转换成DataFrame结构
y_test_user.columns = ['pvalue_level']  # 修改y_test_user的列名为'pvalue_level'
X_test_user.reset_index(drop=True,inplace=True)  # 重置X_test_user索引
# X_test_user.drop('index',axis=1,inplace=True) # 删除上述操作生成的原index索引列
pvalue_null = pd.concat([X_test_user,y_test_user],axis=1)  # 横向合并X_test_user,y_test_user两个表
user = pd.concat([pvalue_nonull,pvalue_null],ignore_index=False)   # 纵向合并pvalue_nonull,pvalue_null表为一个user新表，并重置索引值
user.to_csv('/content/drive/MyDrive/datasets_Aug12/user.csv',index=False,sep=',')  # 将user表数据写入user.csv文件中
user = pd.read_csv('/content/drive/MyDrive/datasets_Aug12/user.csv')  # 读取user数据
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061768 entries, 0 to 1061767
Data columns (total 9 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   userid                 1061768 non-null  int64  
 1   cms_segid              1061768 non-null  int64  
 2   cms_group_id           1061768 non-null  int64  
 3   final_gender_code      1061768 non-null  int64  
 4   age_level              1061768 non-null  int64  
 5   shopping_level         1061768 non-null  int64  
 6   occupation             1061768 non-null  int64  
 7   new_user_class_level   1061768 non-null  float64
 8   pvalue_level           1061768 non-null  float64
dtypes: float64(2), int64(7)
memory usage: 72.9 MB


In [ ]:
ads.fillna(method='pad',inplace=True)
ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846811 entries, 0 to 846810
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   adgroup_id   846811 non-null  int64  
 1   cate_id      846811 non-null  int64  
 2   campaign_id  846811 non-null  int64  
 3   customer     846811 non-null  int64  
 4   brand        846811 non-null  float64
 5   price        846811 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 38.8 MB


<ipython-input-11-1410f221bcef>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ads.fillna(method='pad',inplace=True)


In [ ]:
"""
合并表user、ads、dataset为ads_user_dataset
将合并后的数据写入data.csv中并读取
"""
print(f'dataset表的维度：{dataset.shape}')
print(f'user表的维度：{user.shape}')
print(f'ads表的维度：{ads.shape}')

# 修改表dataset中列名user为userid，以便后面基于主键连接
dataset.rename(columns={'user':'userid'},inplace=True)
dataset.head()

# 将数据集dataset与用户基本信息表user合并，基于主键userid，how='right'表示以右边表为基准连接
user_dataset = pd.merge(user,dataset,on='userid',how='right')
print(f'user_dataset表的维度：{user_dataset.shape}')

# 将数据集与广告基本信息表ads合并，基于主键adgroup_id，how='right'表示以右边表为基准连接
ads_user_dataset = pd.merge(ads,user_dataset,on='adgroup_id',how='right')
print(f'ads_user_dataset表的维度：{ads_user_dataset.shape}')

ads_user_dataset.to_csv('/content/drive/MyDrive/datasets_Aug12/data.csv',index=False,sep=',')
data = pd.read_csv('/content/drive/MyDrive/datasets_Aug12/data.csv')

dataset表的维度：(2000000, 6)
user表的维度：(1061768, 9)
ads表的维度：(846811, 6)
user_dataset表的维度：(2000000, 14)
ads_user_dataset表的维度：(2000000, 19)


In [ ]:
# 获取data每列的缺失值占比
data_null = data.isnull().sum()/len(dataset)*100
data_null = data_null.drop(data_null[data_null==0].index).sort_values(ascending=False)     # 将缺失值占比从高到低排序
missing_data = pd.DataFrame({'Missing Ratio(%)':data_null})
print(f'dataset含有缺失值的属性个数：{len(data_null)}')
print(missing_data)


dataset含有缺失值的属性个数：8
                       Missing Ratio(%)
cms_segid                       5.80955
cms_group_id                    5.80955
final_gender_code               5.80955
age_level                       5.80955
shopping_level                  5.80955
occupation                      5.80955
new_user_class_level            5.80955
pvalue_level                    5.80955


In [ ]:
data.dropna(axis=0, how='any',inplace=True)
# 获取data每列的缺失值占比
data_null = data.isnull().sum()/len(dataset)*100
data_null = data_null.drop(data_null[data_null==0].index).sort_values(ascending=False)     # 将缺失值占比从高到低排序
missing_data = pd.DataFrame({'Missing Ratio(%)':data_null})
print(f'dataset含有缺失值的属性个数：{len(data_null)}')
print(missing_data)

dataset含有缺失值的属性个数：0
Empty DataFrame
Columns: [Missing Ratio(%)]
Index: []


In [ ]:
data.shape

(1883809, 19)

In [ ]:
# 将数据中的时间戳形式转换为日期和时间形式
import datetime
import time
data['time_stamp']=pd.to_datetime(data['time_stamp'],unit='s')
data['time_stamp']
# 从转换后的数据中分别提取：日期、时间、小时，组成新的列
data['date'] = data['time_stamp'].dt.date
data['time'] = data['time_stamp'].dt.time
data['hour'] = data['time_stamp'].dt.hour
# 调整数据集data的列顺序:将'data'、'time'、'hour'这三列数据调至'time_stamp'列后
columns = ['adgroup_id', 'cate_id', 'campaign_id', 'customer', 'brand', 'price',
       'userid', 'cms_segid', 'cms_group_id', 'final_gender_code', 'age_level',
       'shopping_level', 'occupation', 'new_user_class_level ', 'pvalue_level',
       'time_stamp', 'date', 'time', 'hour', 'pid', 'nonclk', 'clk']
data = data[columns]

In [ ]:
# 删除列'time_stamp'、'time'以及'nonclk'
# data = data.drop(['time_stamp','time','nonclk'],axis=1)
data.to_csv('/content/drive/MyDrive/datasets_Aug12/dataNEW.csv',index=False,sep=',')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

data0 = pd.read_csv('/content/drive/MyDrive/datasets_Aug12/dataNEW.csv')
data0.head()

,adgroup_id,cate_id,campaign_id,customer,brand,price,userid,cms_segid,cms_group_id,final_gender_code,age_level,shopping_level,occupation,new_user_class_level,pvalue_level,date,hour,pid,clk
0,1,9025,108570,1337,20169.0,17.0,581738,0.0,8.0,1.0,2.0,3.0,0.0,2.0,1.0,2017-05-07,6,430548_1007,0
1,4,7958,385883,39662,216840.0,85.0,914836,0.0,5.0,2.0,5.0,3.0,0.0,2.0,2.0,2017-05-13,4,430548_1007,0
2,5,7958,385883,39662,216840.0,60.0,914836,0.0,5.0,2.0,5.0,3.0,0.0,2.0,2.0,2017-05-13,4,430548_1007,0
3,8,1626,388290,42349,241359.0,63.0,399907,0.0,5.0,2.0,5.0,3.0,0.0,2.0,1.0,2017-05-09,4,430548_1007,0
4,9,9784,393714,32496,457216.0,396.0,628137,0.0,9.0,1.0,3.0,1.0,0.0,2.0,2.0,2017-05-11,17,430548_1007,0


In [ ]:
clk_counts = data0['clk'].value_counts()

print(clk_counts)

clk
0    1790698
1      93111
Name: count, dtype: int64


In [ ]:
# 统计 clk=0 和 clk=1 的数量
clk_counts = data0['clk'].value_counts()
num_1s = clk_counts[1]  # clk=1 的数量
num_0s = clk_counts[0]  # clk=0 的数量

# 从 clk=0 的样本中随机抽样
data0_clk_0 = data0[data0['clk'] == 0].sample(n=num_1s, random_state=42)

# 获取所有 clk=1 的样本
data0_clk_1 = data0[data0['clk'] == 1]

# 合并这两个 DataFrame
data0_balanced = pd.concat([data0_clk_0, data0_clk_1])

# 查看平衡后的结果
print(data0_balanced['clk'].value_counts())

clk
0    93111
1    93111
Name: count, dtype: int64


In [ ]:
X = data0[['price', 'age_level', 'shopping_level', 'occupation', 'pvalue_level', 'hour', 'final_gender_code', 'new_user_class_level ']]
y = data0['clk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

theta_star = model.coef_
print("Theta Star:")
print(theta_star)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Theta Star:
[[-1.61163910e-08  7.48190052e-12 -3.85740233e-12 -5.32501344e-13
  -2.11118120e-12 -1.72834233e-11  2.25629922e-12  4.66983204e-13]]
Accuracy: 0.9502577223817689
Classification Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.97    358021
           1       0.00      0.00      0.00     18741

    accuracy                           0.95    376762
   macro avg       0.48      0.50      0.49    376762
weighted avg       0.90      0.95      0.93    376762



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
